In [ ]:
import openai
import os
import support

#from nltk.tokenize import sent_tokenize
#import re
#import nltk
#import backup_support

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv("OPENAI_API_KEY")

m = open('QuestionDatabase.txt', 'w')
m.close()


In [ ]:
def get_completion_from_messages(messages, model = 'gpt-3.5-turbo', temperature = 0, top_p = 0.9, max_tokens = 500):
    response = openai.ChatCompletion.create(
        messages = messages,
        model = model,
        temperature = temperature,
        top_p = top_p,
        max_tokens = max_tokens,
    )
    
    return response.choices[0].message['content']

In [ ]:
summarised = {
  "1": "Madagascar's forests are being converted to agricultural land due to insect pests destroying crops, leading to habitat and biodiversity loss, but some insectivorous bats are thriving and providing pest control services.",
  "2": "A study by zoologist Ricardo Rocha shows that bats are helping rice farmers in Madagascar by eating insects, which can reduce the need for deforestation.",
  "3": "Madagascar is an important region for bat conservation, with one-fifth of all mammal species being bats.",
  "4": "Several species of indigenous bats in Madagascar are taking advantage of habitat modification to hunt insects above rice fields.",
  "5": "Six species of bat in Madagascar are preying on rice pests, which cause financial pressure on farmers and encourage deforestation.",
  "6": "A study investigated the feeding activity of bats in farmland bordering a national park in Madagascar.",
  "7": "Researchers used ultrasonic recorders and DNA barcoding techniques to study bat feeding habits in different sites.",
  "8": "Bats preferentially forage in rice fields due to lack of water and nutrient run-off, and DNA analysis showed they fed on economically important insect pests.",
  "9": "The study is the first to show bats acting as pest controllers in Madagascar, benefiting both farmers and conservationists.",
  "10": "Malagasy bats also feed on disease-carrying mosquitoes and blackflies, providing additional benefits to local people.",
  "11": "The relationship between bats and local people in Madagascar is complicated, as bats are a source of protein but can also be seen as unclean, yet they are associated with sacred caves and the ancestors.",
  "12": "Maximizing bat populations can help boost crop yields and promote sustainable livelihoods, and further research is needed to quantify this contribution."
}

m = open('QuestionDatabase.txt', 'a')
m.write(f"""summarised: {summarised}""")
m.close()

In [ ]:
f = open('Passage.txt', 'r')

questions = support.extract_questions(f.read())


questions_list = list(questions.values())


question = questions_list[2]

question_list = {'question': question}


In [ ]:
m = open('QuestionDatabase.txt', 'a')
m.write(f"""\n\n question_list = {str(question_list)}""")
m.close()

In [ ]:

#for info in sample.values():
infos = support.extract_infos_statement(question)
print(infos, "\n")

info = {}
info['actor'] = infos['actor']
info['acted upon'] = infos['acted upon']

print(info)


#Change the format into JSON for clean visuals
#JSON format outputs

In [ ]:
MCQ_questions_for_statement = support.generate_questions_for_statement(infos, question, summarised)
print(MCQ_questions_for_statement)
#Instead of picking word, run loop through dict, avoid those with None

#Find out ways to prevent hallucinations
#Turn it into a flow

In [ ]:
m = open('QuestionDatabase.txt', 'a')
m.write(f"""\n\n MCQ_questions_for_statements: {MCQ_questions_for_statement}""")
m.close()


In [ ]:
location = support.output_to_string(support.find_location_of_answers(summarised, infos))
print(location)
question_location = support.output_to_string(support.generate_questions_for_locations(infos, summarised[location["paragraph"]]))
print(question_location)

question_for_statement = {
    'question': question_location['question'],
    'answer': location['paragraph'],
    'explanation': location['explanation']
}

In [ ]:
m = open('QuestionDatabase.txt', 'a')
m.write(f"""\n\n question_for_statement: {question_for_statement}""")
m.close()

In [ ]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    if prompt != '':
        context.append({'role':'user', 'content':f"{prompt}"})
        panels.append(
            pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
        response = get_completion_from_messages(context) 
        context.append({'role':'assistant', 'content':f"{response}"})

        panels.append(
            pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [ ]:
import panel as pn
pn.extension()

panels = []
m = open('QuestionDatabase.txt', 'r')

context = [
    {'role': 'system', 'content': """
You are an IELTS Reading assistant who will guide the user through a given question in the QuestionDatabase.txt. \
You will be given a file. Here are what you should do:
- Step 1: Find a dictionary called "question_list", then ask the user if the question_list matches the questions the user input.\
Wait for user input.

- Step 2: Then, mention that you will go through each question with the user.

- Step 3: Find a dictionary called "MCQ_questions_for_statement" and ask the user the questions one by one.\
For each question, If the user input the correct answer, congratulate the user and move on to the next question.
If the user input the wrong answer, explain why it's wrong without mentioning the correct answer, and let the user input again.
Do not continue until the user gets the right option.

- Step 4: Find a dictionary called "question_for_statement". Find the key "question" in it and ask the user.
Wait for the user input.
Then, check the user's input with the value in the key 'answer' in the dictionary.
If they are similar, congratulate the user and move on to the next step.
If they are not similar, explain why the user is wrong and let the user input again.
Do not continue until the user gives the right answer.


"""},
    {'role': 'user', 'content': m.read()},
]

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard   